In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
import numpy as np

df = pl.DataFrame(
    {"x": np.random.normal(size=100_000)}).with_row_index().with_columns(pl.lit(1).alias("const")
)

kernel = np.array([1] * 50)

In [ ]:
df.select(pds.convolve("x", kernel, method="fft"))


In [ ]:
from scipy.signal import convolve

In [ ]:
arr1 = df["x"].to_numpy()
arr2 = kernel

In [ ]:
convolve(arr1, arr2)

In [ ]:
%timeit df.select(pds.convolve("x", kernel))
%timeit df.select(pds.convolve("x", kernel, method="fft"))
%timeit convolve(arr1, arr2)

In [ ]:
df = pds.random_data(size=100_000, n_cols = 0).select(
    pds.random_int(0, 200).alias("x"),
    pds.random_int(0, 200).alias("y"),
    pl.Series([1] * 50_000 + list(range(50_000, 100_000))).alias("test")
)
df.head()

In [ ]:
df.select(
    pl.col("x").qcut(10, left_closed=False, allow_duplicates=True, include_breaks=True)
        .struct.field("brk")
        .value_counts()
        .sort()
).unnest("brk")

In [ ]:
df.select(
    pl.corr("x", "y")
)

In [ ]:

df.select(
    pds.kendall_tau("x", "y")
)

In [ ]:
from scipy.stats import kendalltau

x = df["x"].to_numpy()
y = df["y"].to_numpy()

In [ ]:
%%timeit
kendalltau(x,y, nan_policy="omit")

In [ ]:
df.sort(pl.col("x").rank(method="random")).select(
    "x",
    "y",
    pl.col("y").rank(method="max").cast(pl.Float64).alias("r"),
    (-pl.col("y")).rank(method="max").cast(pl.Float64).alias("l"),
).with_columns(
    pl.col("r").diff().abs().alias("r_abs_diff"),
    (pl.col("l") * (pl.len() - pl.col("l"))).alias("l(n-l)"),
).select(
    1 - (pl.len() / 2) * (pl.col("r_abs_diff").sum() / pl.col("l(n-l)").sum())
)